In [6]:
# !pip install fasttext

In [13]:
import os
import time
import csv
import numpy as np
import pandas as pd
import fasttext
import gensim
import sklearn.metrics
import re
import unicodedata
from tqdm.auto import tqdm
import sys

csv.field_size_limit(sys.maxsize) # to avoid error: _csv.Error: field larger than field limit (131072)

9223372036854775807

In [12]:

# download the training set if it does not exist
if not os.path.isfile("github-labels-top3-803k-train.csv"):
  !curl "https://tickettagger.blob.core.windows.net/datasets/github-labels-top3-803k-train.tar.gz" | tar -xz

display(pd.read_csv("github-labels-top3-803k-train.csv", nrows=5))

if not os.path.isfile("github-labels-top3-803k-test.csv"):
  !curl "https://tickettagger.blob.core.windows.net/datasets/github-labels-top3-803k-test.tar.gz" | tar -xz

display(pd.read_csv("github-labels-top3-803k-test.csv", nrows=5))

,Unnamed: 0,issue_url,issue_label,issue_created_at,issue_author_association,repository_url,issue_title,issue_body
0,0,https://api.github.com/repos/eamodio/vscode-gi...,bug,2021-01-02T18:07:30Z,NONE,https://api.github.com/repos/eamodio/vscode-gi...,Welcome screen on every editor window is very ...,I just discovered Gitlens and find the functio...
1,1,https://api.github.com/repos/binwiederhier/pco...,bug,2020-12-31T18:19:31Z,OWNER,https://api.github.com/repos/binwiederhier/pcopy,"""pcopy invite"" and ""pcopy paste abc:"" does not...",NaN
2,2,https://api.github.com/repos/binwiederhier/pco...,bug,2021-01-03T04:33:36Z,OWNER,https://api.github.com/repos/binwiederhier/pcopy,"UI: Modal overlay is half transparent, shouldn...",NaN
3,3,https://api.github.com/repos/Sothatsit/RoyalUr...,enhancement,2020-12-25T00:46:00Z,OWNER,https://api.github.com/repos/Sothatsit/RoyalUr...,Make the loading screen scale with browser win...,Currently the loading wheel is a fixed size in...
4,4,https://api.github.com/repos/Malivil/TTT-Custo...,bug,2021-01-02T21:36:57Z,OWNER,https://api.github.com/repos/Malivil/TTT-Custo...,Spectator - Investigate a way to strip weapons...,To bring magneto stick floating


,Unnamed: 0,issue_url,issue_label,issue_created_at,issue_author_association,repository_url,issue_title,issue_body
0,6,https://api.github.com/repos/tlnagy/TIFF.jl/is...,enhancement,2020-04-07T09:08:50Z,NONE,https://api.github.com/repos/tlnagy/TIFF.jl,"ERROR: KeyError: key (TIFF.SAMPLEFORMAT_INT, 0...",One more error might need to be caught.\r\n`4D...
1,19,https://api.github.com/repos/tisboyo/Twitch_Bo...,enhancement,2020-11-27T07:17:21Z,OWNER,https://api.github.com/repos/tisboyo/Twitch_Bot,Add database backup to dropbox,NaN
2,25,https://api.github.com/repos/DrWhoCares/imgdan...,enhancement,2021-01-02T19:35:34Z,OWNER,https://api.github.com/repos/DrWhoCares/imgdanke,Add a button/method to open the Source or Outp...,Could also add a method to open up path to eac...
3,30,https://api.github.com/repos/DrWhoCares/imgdan...,bug,2021-01-02T20:55:34Z,OWNER,https://api.github.com/repos/DrWhoCares/imgdanke,Processes are being started twice,At some point I refactored a few things and en...
4,54,https://api.github.com/repos/Bean-1/AOT/issues/3,bug,2020-12-29T15:34:35Z,OWNER,https://api.github.com/repos/Bean-1/AOT,Cannot add hp to wall,NaN


'type'

In [36]:

def count_tokens(text):
	return text.count(" ") + 1

def preprocess(text):
  text = str(text)

  # lowercase
  # text = text.lower()
  
  # remove html tags
  # text = gensim.parsing.preprocessing.strip_tags(text)
  
  # remove punctuation
  # text = gensim.parsing.preprocessing.strip_punctuation(text)
  
  # remove numerics
  # text = gensim.parsing.preprocessing.strip_numeric(text)
  
  # remove consecutive whitespace characters and convert tabs to spaces
  text = gensim.parsing.preprocessing.strip_multiple_whitespaces(text)
  
  # text = gensim.parsing.preprocessing.strip_short(text, minsize=3)
  
  # text = gensim.parsing.preprocessing.remove_stopwords(text)
  
  # text = gensim.parsing.preprocessing.stem_text(text)
  
  return text

def preprocess_row(row):
  doc = row["issue_author_association"].lower()
  doc += " " + preprocess(row["issue_title"])
  doc += " " + preprocess(row["issue_body"])

  return doc

In [37]:

def transform_to_fasttext_format(i_path, o_path):
	with open(i_path, "r") as i_f, open(o_path, "w") as o_f:
		reader = csv.DictReader(i_f)
		for row in tqdm(reader, desc="Transform to fastText format"):
			o_f.write(f"__label__{row['issue_label']} {preprocess_row(row)}\n")

transform_to_fasttext_format("github-labels-top3-803k-train.csv", "issues.train")

!head -n 5 "issues.train"

Transform to fastText format: 0it [00:00, ?it/s]

__label__bug none Welcome screen on every editor window is very tedious I just discovered Gitlens and find the functionality useful, thank you to all who contribute. I have about a dozen editor windows open, and the install process added a Gitlens welcome tab to each and every one of them. Combined with the snowflake effect, all of the sudden VScode was consuming 300-400% cpu and my fan was raging, as soon as I hunted them all down everything was back to fine. The welcome note content is great (although putting it on _all_ the windows is a bit much, don't know how much control you have on that). But overall it was a bit of a sour first-use experience, just wanted to provide that feedback.
__label__bug owner "pcopy invite" and "pcopy paste abc:" does not check if clipboard exists 
__label__bug owner UI: Modal overlay is half transparent, shouldn't be 
__label__enhancement owner Make the loading screen scale with browser window size Currently the loading wheel is a fixed size in pixels, 

In [38]:
# https://fasttext.cc/docs/en/python-module.html#train_supervised-parameters
model = fasttext.train_supervised("issues.train", 
                                  minCount=30,
                                  # autotuneValidationFile="issues.valid",
                                  # autotuneDuration=5*60 # 5 minutes
                                  )
# model.quantize()
# model.save_model(f"github-labels-{int(time.time())}.ftz")

Read 78M words
Number of words:  79487
Number of labels: 3
Progress: 100.0% words/sec/thread: 1786941 lr:  0.000000 avg.loss:  0.512125 ETA:   0h 0m 0s


In [39]:
# confusion matrix
y_true = []
y_pred = []

with open("github-labels-top3-803k-test.csv") as f:
  reader = csv.DictReader(f)
  for row in tqdm(reader, desc="Benchmarking Inference Performance"):
    pred = model.predict(preprocess_row(row))[0][0][9:]
    y_true.append(row["issue_label"])
    y_pred.append(pred)

for label in ["bug", "enhancement", "question"]:
# for label in ["bug", "feature", "question", "documentation"]:
  P_c = sklearn.metrics.precision_score(y_true, y_pred, average=None, labels=[label])[0]
  R_c = sklearn.metrics.recall_score(y_true, y_pred, average=None, labels=[label])[0]
  F1_c = sklearn.metrics.f1_score(y_true, y_pred, average=None, labels=[label])[0]
  print(f"=*= {label} =*=")
  print(f"precision:\t{P_c:.4f}")
  print(f"recall:\t\t{R_c:.4f}")
  print(f"F1 score:\t{F1_c:.4f}")
  print()


P = sklearn.metrics.precision_score(y_true, y_pred, average='micro')
R = sklearn.metrics.recall_score(y_true, y_pred, average='micro')
F1 = sklearn.metrics.f1_score(y_true, y_pred, average='micro')

print("=*= global =*=")
print(f"precision:\t{P:.4f}")
print(f"recall:\t\t{R:.4f}")
print(f"F1 score:\t{F1:.4f}")

Benchmarking Inference Performance: 0it [00:00, ?it/s]

=*= bug =*=
precision:	0.8369
recall:		0.8734
F1 score:	0.8548

=*= enhancement =*=
precision:	0.8232
recall:		0.8436
F1 score:	0.8333

=*= question =*=
precision:	0.6490
recall:		0.4107
F1 score:	0.5031

=*= global =*=
precision:	0.8207
recall:		0.8207
F1 score:	0.8207
